# Dataset Preparation for First ICU Prediction

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math
import numpy as np
import pandas as pd
import spacy

import seaborn as sns
sns.set(style = 'darkgrid')

import matplotlib.pyplot as plt
%matplotlib inline

from pathlib import Path

In [ ]:
path = Path('./data')
raw_csv = path/'mimic_icu_pred_raw_dataset.csv'
proc_csv = path/'mimic_icu_pred_proc_dataset.csv'
fig_dir = path/'workdir/figures'

## Functions

In [ ]:
def get_sample(df, sample_pct=0.01, with_val=True, seed=None):
  train = df.loc[(df['split']) == 'train'].sample(frac=sample_pct, random_state=seed)
  train.reset_index(inplace=True, drop=True)

  if with_val:
    val = df.loc[(df['split']) == 'val'].sample(frac=sample_pct, random_state=seed)
    val.reset_index(inplace=True, drop=True)
    return pd.concat([train, val], axis=0) 

  return train

In [ ]:
def tokenize_text(text):
  tokens = [token.text for token in nlp(text)]
  return ' '.join(tokens)

def group_eth(eth):
  eth = eth.lower()
  if 'white' in eth:
    return 'white'
  elif 'black' in eth:
    return 'black'
  elif 'hispanic' in eth:
    return 'hispanic'
  elif 'asian' in eth:
    return 'asian'
  else:
    return 'unknown'

## Data Processing

1. Load in the data
2. Drop duplicates
3. Merge `category`, `description`, and `text` into a new column called `note`
4. Tokenize text using `scispacy` and create new column called `scispacy_note` to save tokenized text
5. Save a csv file onto disk

In [ ]:
df = pd.read_csv(raw_csv)
df.shape

In [ ]:
nlp = spacy.load('en_core_sci_md', disable=['parser', 'ner', 'tagger'])

df.drop_duplicates(inplace=True)
df['note'] = df['category'].str.cat(df['description'], sep='\n')
df['note'] = df['note'].str.cat(df['text'], sep='\n')
df['ethnicity'] = df['ethnicity'].apply(group_eth)
df['scispacy_note'] = df['note'].apply(tokenize_text)
df.shape

In [ ]:
df.to_csv(proc_csv, index=False)

## Data Analysis

In [ ]:
df = pd.read_csv(proc_csv)

In [ ]:
df.columns

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

sns.distplot(df['icu_los'], kde=False, ax=ax, bins=80)
ax.set_xlabel('ICU Length of Stay (days)')
ax.set_ylabel('# notes')
ax.set_xlim(0, 40)
ax.xaxis.labelpad = 20
ax.yaxis.labelpad = 20
fig.savefig(fig_dir/'icu_los_hist.tif', dpi=300)
# ax.text(ax.get_xlim()[1]*0.50, ax.get_ylim()[1]*0.80, f"Min: {df['icu_los'].min()}, Avg: {df['icu_los'].mean(): 0.2f}, Max: {df['icu_los'].max()}", fontsize=15, ha='center', va='bottom')

## Dataset Processing

In [ ]:
df = pd.read_csv(path/'raw_dataset.csv')
df.drop_duplicates(inplace=True)
df['note'] = df['category'].str.cat(df['description'], sep='\n')
df['note'] = df['note'].str.cat(df['text'], sep='\n')
df['ethnicity'] = df['ethnicity'].apply(group_eth)
df['scispacy_note'] = df['note'].apply(tokenize_text)
df.head()

In [ ]:
df.to_csv(path/'processed_dataset.csv', index=False)

In [ ]:
full_df = pd.read_csv(path/'full_raw_dataset.csv')

full_df.drop_duplicates(inplace=True)
full_df['note'] = full_df['category'].str.cat(full_df['description'], sep='\n')
full_df['note'] = full_df['note'].str.cat(full_df['text'], sep='\n')
full_df['ethnicity'] = full_df['ethnicity'].apply(group_eth)
full_df['scispacy_note'] = full_df['note'].apply(tokenize_text)
print(full_df.shape)
full_df.head()

In [ ]:
full_df.to_csv(path/'full_processed_dataset.csv', index=False)

## Stats

In [ ]:
df = pd.read_csv(path/'processed_dataset.csv')

In [ ]:
print(f"{(len(df.loc[df['class_label'] == 1])/len(df)):0.3f}")

In [ ]:
from utils.splits import set_two_splits

In [ ]:
avg_p = []

for seed in range(127, 227):
  sdf = set_two_splits(df.copy(), name='test', seed=seed)
  test_size = len(sdf.loc[(sdf['split'] == 'test')])
  test_pos = len(sdf.loc[(sdf['split'] == 'test') & (sdf['class_label'] == 1)])
  avg_p.append(test_pos/test_size)  

In [ ]:
avg_p = np.array(avg_p)

In [ ]:
print(f"Average prevalence = {(avg_p.mean()):0.3f}")

In [ ]:
print(f"Std prevalance = {(avg_p.std()):0.3f}")

In [ ]:
df.loc[(df['class_label'] == 0)].groupby('category').apply(lambda g: pd.Series(g['subject_id'].nunique()))

In [ ]:
df.loc[(df['class_label'] == 0)].groupby('category').size()